# Azure Cognitive Search Llamaindex Vector Code Sample
This code demonstrates how to use Azure Cognitive Search with OpenAI and the Azure Cognitive Search Llamaindex Vector Store.
To run the code, install the following packages. Please use the latest pre-release version `pip install azure-search-documents --pre`.

In [ ]:
! pip install azure-search-documents --pre
! pip install openai
! pip install llama-index
! pip install python-dotenv

## Import required libraries and environment variables

In [91]:
# Import required libraries
import os
from dotenv import load_dotenv
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from llama_index.vector_stores.cogsearch import (
    IndexManagement,
    CognitiveSearchVectorStore,
)
from langchain.embeddings import OpenAIEmbeddings
from llama_index.llms import AzureOpenAI
from llama_index import (
    LangchainEmbedding,
    SimpleDirectoryReader,
    StorageContext,
    ServiceContext,
    VectorStoreIndex,
)


## Configure OpenAI settings

In [92]:
# Configure environment variables  
load_dotenv()  
openai.api_type: str = "azure"  
openai.api_key = os.getenv("AZURE_OPENAI_API_KEY")  
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")  
openai.api_version = os.getenv("AZURE_OPENAI_API_VERSION")  
llm_deployment_name: str = "text-davinci-003"
embedding_deployment_name: str = "text-embedding-ada-002"

## Configure Vector Store settings

In [93]:
key: str = os.getenv("AZURE_SEARCH_ADMIN_KEY") 
cognitive_search_credential = AzureKeyCredential(key)
service_endpoint: str = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
index_name: str = "llamaindex-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=service_endpoint,
    credential=cognitive_search_credential,
)

# Use search client to demonstration using existing index
search_client = SearchClient(
    endpoint=service_endpoint,
    index_name=index_name,
    credential=cognitive_search_credential,
)

## Create embeddings and vector store instances
Read your data, generate OpenAI embeddings and export to a format to insert your Azure Cognitive Search index:

In [94]:
metadata_fields = {}  # No additional metadata fields in this example  
vector_store = CognitiveSearchVectorStore(  
    search_or_index_client=index_client,  
    index_name=index_name,  
    filterable_metadata_field_keys=metadata_fields,  
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,  
    id_field_key="id",  
    chunk_field_key="content",  
    embedding_field_key="content_vector",  
    metadata_string_field_key="metadata",  
    doc_id_field_key="doc_id",  
)  

In [95]:
llm = AzureOpenAI(engine=llm_deployment_name, model="text-davinci-003")

embed_model = LangchainEmbedding(
    OpenAIEmbeddings(
        model="text-embedding-ada-002",
        deployment=embedding_deployment_name,
        openai_api_key=openai.api_key,
        openai_api_base=openai.api_base,
        openai_api_type=openai.api_type,
        openai_api_version=openai.api_version,
    ),
    embed_batch_size=1,
)

# load documents
documents = SimpleDirectoryReader(
    "../data/sample-data"
).load_data()

storage_context = StorageContext.from_defaults(vector_store=vector_store)
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, service_context=service_context
)

## Perform a vector similarity search

In [96]:
query_engine = index.as_query_engine()
response = query_engine.query("What did the president say about Ketanji Brown Jackson")
print(response)

 The president said that Ketanji Brown Jackson is one of the nation's top legal minds, a former top litigator in private practice, a former federal public defender, and from a family of public school educators and police officers. He also said that she is a consensus builder and has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans.
